In [ ]:
import IPython
import sys

def clean_notebook():
    IPython.display.clear_output(wait=True)
    print("Notebook cleaned.")

!pip install --upgrade evidently==0.7.14

# Clean up the notebook
clean_notebook()

In [ ]:
from evidently import Report
from evidently.metrics import *
from evidently.presets import *
import pandas as pd
import numpy as np

# Create mockup DataFrame
np.random.seed(42)

df = pd.DataFrame({
    'age': np.random.randint(18, 70, 1000),
    'income': np.random.randint(20000, 150000, 1000),
    'credit_score': np.random.randint(300, 850, 1000),
    'loan_amount': np.random.randint(5000, 50000, 1000),
    'employment_years': np.random.randint(0, 30, 1000),
    'debt_ratio': np.random.uniform(0, 1, 1000),
    'num_credit_lines': np.random.randint(1, 10, 1000),
    'category': np.random.choice(['A', 'B', 'C', 'D'], 1000),
    'approved': np.random.choice([0, 1], 1000, p=[0.3, 0.7])
})


# reference = ข้อมูล train
reference = df.sample(500, random_state=42)

# current = ข้อมูลใหม่ที่เข้าโมเดล
current = df.sample(500, random_state=99)


print(f"DataFrame shape: {df.shape}")
df.head(10)


#### Create Data dfift

In [ ]:
# =========================
# 2) Inject Drift in 2 Columns
#    - Numeric drift:  income  (ขยับค่าเฉลี่ย/สเกลให้สูงขึ้น)
#    - Categorical drift: category (บิดสัดส่วนให้ skew มากขึ้น)
# =========================

 # Income drift (shift & scale), แล้ว clip ให้อยู่ในช่วงสมเหตุสมผล
current['income'] = (current['income'] * 1.6 + 10_000).clip(lower=15_000, upper=300_000).astype(int)

#  Category drift (เปลี่ยน distribution ให้ 'D' เด่นขึ้น)
new_cats = np.random.choice(['A', 'B', 'C', 'D'], size=len(current), p=[0.10, 0.10, 0.10, 0.70])
current['category'] = new_cats



In [ ]:

# สร้างรายงาน Data Drift
drift_report = Report([
    DataDriftPreset()
])

my_eval = drift_report.run(reference,current)
my_eval.save_html("data_drift_report.html")



In [ ]:
metrics = [
            DataDriftPreset(['age','income','credit_score','category'])
    ]

drift_report = Report(metrics)
result = drift_report.run(reference_data=reference, current_data=current)

# Save HTML
result